## Import module

In [142]:
from textblob import TextBlob
import math
from collections import Counter, defaultdict
import tweepy as tw
import json
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
%matplotlib inline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

## Set up token API

In [143]:
api_key = "HAcIeikl6eRheQP1oKYGDwPx0"
api_secret = "gEWmqG7QVXzudnviUXDo98L2UKGYg9PNqy1bAR1geClkHjqBGk"
access_token = "1481095074465579008-1pApYlU4HRJWpbAXcbdVEVGzdACuEz"
access_token_secret = "htgD81H4mOxXw5MglAljFRHhcfGAuwsNomXVqvGw0Obwe"
auth = tw.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth,wait_on_rate_limit=True)

## Crawl data

In [144]:
number_of_tweets = 100
tweets = []
user_id =[]
created_at = []
keyword_list = ['News', 'Stock'] # , 'Customers', 'Employees', 'Company', 'Price', 'Shareholders', 'Market', 'Soaring', 'Trends'
company = 'Amazon'
final_kw_list = []

# Crawl data from keywords
for i in range (len(keyword_list)):
    kw = str(company + ' ' + keyword_list[i])
    print(kw)
    final_kw_list.append(kw)
    for i in tw.Cursor(api.search, q = kw , lang ="en", since = "2022-01-14",tweet_mode = "extended").items(number_of_tweets):
        tweets.append(i.full_text)
        user_id.append(i.user.id)
        created_at.append(i.created_at)

Amazon News
Amazon Stock


## Dataframe

In [145]:
df = pd.DataFrame({"Created_at": created_at,"User_id": user_id,"Tweets":tweets})
df

,Created_at,User_id,Tweets
0,2022-02-07 05:27:43,781709113713684480,RT @BBCBusiness: Amazon and Nike exploring Pel...
1,2022-02-07 05:26:32,109064246,RT @JoshuaPotash: Here's the full article. And...
2,2022-02-07 05:26:30,1405283490422087683,RT @JoshuaPotash: Here's the full article. And...
3,2022-02-07 05:25:15,811414973985550336,RT @JoshuaPotash: Here's the full article. And...
4,2022-02-07 05:25:01,32518396,RT @OccupyDemocrats: If you’re a Democrat who ...
...,...,...,...
195,2022-02-07 03:54:33,1404406896220676099,GeForce RTX 3070 - PNY - XLR8 Gaming REVEL is ...
196,2022-02-07 03:52:37,499672969,(US)In Stock: XFX Speedster QICK210 Radeon RX ...
197,2022-02-07 03:52:20,1377519267143376898,Xbox Series S is now in stock. \nATC: https://...
198,2022-02-07 03:52:00,36293735,Amazon and Nike are considering making a bid f...


## Clean data

In [146]:
df = df[~df.Tweets.str.contains("RT")]
df = df.reset_index(drop = True)

In [147]:
def clean_up_tweet(text):
    text = re.sub(r'@[A-Za-z0-9_]+','',text) # Remove mentions @
    text = re.sub(r'#', '', text) # Remove hastag symbol
    text = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', text) # Remove hyper link
    text = re.sub('[^A-Za-z0-9]+', ' ', text) # Remove special characters
    text = re.sub(r'[0-9]+', '', text)
    #Remove stop words
    res = ''
    for word in text.split():
        if word not in stopwords.words('english'):
            res += WordNetLemmatizer().lemmatize(word) + ' '

    return res

In [148]:
import string
from nltk import TweetTokenizer, PorterStemmer


#Preprocessing tweets
def process_tweet(tweet):

    # instantiate tokenizer class
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)

    # tokenize tweets
    tweet_tokens = tokenizer.tokenize(tweet)

    #Import the english stop words list from NLTK
    stopwords_english = stopwords.words('english')

    #Creating a list of words without stopwords
    tweets_clean = []
    for word in tweet_tokens:
        if word not in stopwords_english and word not in string.punctuation:
            tweets_clean.append(word)

    # Instantiate stemming class
    stemmer = PorterStemmer()

    # Creating a list of stems of words in tweet
    tweets_stem = []
    for word in tweets_clean:
        stem_word = stemmer.stem(word)
        tweets_stem.append(stem_word)

    return tweets_clean

In [149]:
from nltk.sentiment import SentimentIntensityAnalyzer

df['Tweets'] = df['Tweets'].apply(clean_up_tweet)
df = df.drop_duplicates(subset='Tweets')
df = df.reset_index(drop=True)
df['Word_list'] = df['Tweets'].apply(process_tweet)

sentiments = SentimentIntensityAnalyzer()
df['Compound'] = [sentiments.polarity_scores(i)["compound"] for i in df["Tweets"]]
score = df["Compound"].values
sentiment = []
for i in score:
    if i >= 0.05 :
        sentiment.append(1)
    elif i <= -0.05 :
        sentiment.append(0)
    else:
        sentiment.append('Neutral')
df["Sentiment"] = sentiment
df.drop(df.loc[df['Sentiment']=='Neutral'].index, inplace=True)
df = df.reset_index(drop=True)
df

,Created_at,User_id,Tweets,Word_list,Compound,Sentiment
0,2022-02-07 05:24:27,1267197215635726336,lt Ethereum Price gt ETH USDT Binance Check Pr...,"[lt, ethereum, price, gt, eth, usdt, binance, ...",0.9042,1
1,2022-02-07 05:20:37,730176654,Amazon India sign MoU turn rural woman entrepr...,"[amazon, india, sign, mou, turn, rural, woman,...",0.1779,1
2,2022-02-07 05:20:03,1267197215635726336,lt Binance Coin Price gt BNB USDT Binance Chec...,"[lt, binance, coin, price, gt, bnb, usdt, bina...",0.9042,1
3,2022-02-07 05:19:40,1287568658,FB venue share family news Spotify Rogan preju...,"[fb, venue, share, family, news, spotify, roga...",-0.1779,0
4,2022-02-07 05:12:00,867593107956654080,Watch Whitestone Dome display protector return...,"[watch, whitestone, dome, display, protector, ...",0.5994,1
...,...,...,...,...,...,...
61,2022-02-07 04:09:41,1418587410384908293,US In Stock PowerColor Hellhound AMD Radeon RX...,"[us, stock, powercolor, hellhound, amd, radeon...",0.3400,1
62,2022-02-07 04:04:53,1418587410384908293,US In Stock XFX Speedster SWFT Radeon RX CORE ...,"[us, stock, xfx, speedster, swft, radeon, rx, ...",0.3400,1
63,2022-02-07 03:59:57,3396302147,Peloton stock shoot higher report Amazon acqui...,"[peloton, stock, shoot, higher, report, amazon...",0.3182,1
64,2022-02-07 03:52:00,36293735,Amazon Nike considering making bid Peloton Pel...,"[amazon, nike, considering, making, bid, pelot...",0.3400,1


## Building Frequency dictionary

In [150]:
#Frequency generating function
def build_frequency(tweets, ys):
    yslist = np.squeeze(ys).tolist()
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            freqs[pair] = freqs.get(pair, 0) + 1

    return freqs

In [168]:
freqs = build_frequency(df['Tweets'], df['Sentiment'])
freqs

{('lt', 1): 16,
 ('ethereum', 1): 5,
 ('price', 1): 18,
 ('gt', 1): 18,
 ('eth', 1): 6,
 ('usdt', 1): 8,
 ('binance', 1): 17,
 ('check', 1): 8,
 ('headline', 1): 8,
 ('news', 1): 19,
 ('cedi', 1): 2,
 ('osman', 1): 2,
 ('kevin', 1): 2,
 ('love', 1): 6,
 ('team', 1): 3,
 ('lead', 1): 2,
 ('cavaliers', 1): 2,
 ('biggest', 1): 2,
 ('comeback', 1): 2,
 ('win', 1): 2,
 ('year', 1): 2,
 ('pacers', 1): 2,
 ('bi', 1): 1,
 ('amazon', 1): 41,
 ('india', 1): 2,
 ('sign', 1): 1,
 ('mou', 1): 1,
 ('turn', 1): 1,
 ('rural', 1): 1,
 ('woman', 1): 1,
 ('entrepreneur', 1): 2,
 ('karnataka', 1): 1,
 ('via', 1): 1,
 ('binaryic', 1): 1,
 ('marketplace', 1): 1,
 ('coin', 1): 3,
 ('bnb', 1): 6,
 ('fb', 0): 1,
 ('venue', 0): 1,
 ('share', 0): 1,
 ('family', 0): 1,
 ('news', 0): 6,
 ('spotify', 0): 1,
 ('rogan', 0): 1,
 ('prejudicial', 0): 1,
 ('hol', 0): 1,
 ('get', 0): 2,
 ('music', 0): 1,
 ('podcasts', 0): 1,
 ('amazon', 0): 19,
 ('watch', 1): 1,
 ('whitestone', 1): 1,
 ('dome', 1): 5,
 ('display', 1): 1,


## Train model

In [169]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df= 0.9, min_df=2, max_features=1000, stop_words='english')

# Bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(df['Tweets'])

print(bow_vectorizer.get_feature_names())
# Associate the indices with each unique word
print(bow_vectorizer.vocabulary_)
# Print the numerical feature vector
print(bow.toarray())

['amazon', 'amd', 'amp', 'apple', 'associate', 'asus', 'atc', 'bbc', 'best', 'bid', 'big', 'biggest', 'binance', 'binancepay', 'bnb', 'book', 'bucks', 'buschlightclash', 'buy', 'card', 'cassie', 'cavaliers', 'cedi', 'change', 'check', 'climate', 'coin', 'comeback', 'coming', 'companies', 'company', 'convoy', 'day', 'deal', 'devil', 'dome', 'earn', 'earthquake', 'echo', 'edition', 'eth', 'ethereum', 'euphoria', 'exaggerating', 'exploring', 'far', 'fortune', 'galvanizes', 'gaming', 'gb', 'glass', 'going', 'good', 'google', 'got', 'graphics', 'gt', 'headline', 'hell', 'ikea', 'illegal', 'including', 'kevin', 'kindle', 'latest', 'lead', 'like', 'link', 'list', 'love', 'lt', 'mary', 'mattthomas', 'membership', 'money', 'month', 'msi', 'new', 'news', 'nike', 'offer', 'open', 'osman', 'ottawa', 'pacers', 'page', 'peloton', 'permission', 'pick', 'politico', 'potential', 'powercolor', 'price', 'prices', 'prime', 'product', 'progress', 'protector', 'ps', 'purchase', 'qick', 'qualifying', 'radeon

C:\Users\hlinh\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [162]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_df= 0.9, min_df=2, max_features=1000, stop_words='english')

# TFIDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(df['Tweets'])

In [163]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

y = df['Sentiment'].values.astype('float')

X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(bow, y, random_state=42, test_size=0.2)
lreg = LogisticRegression()
lreg.fit(X_train_bow, y_train_bow) # training the model
prediction = lreg.predict_proba(X_test_bow) # predicting on the validation set
prediction_float = prediction[:,1] >= 0.3 # if prediction >= 0.2 then 1 or else 0
prediction_float = prediction_float.astype('float')
f1_score(y_test_bow, prediction_float) # calculate f1 score

0.923076923076923

In [164]:
from sklearn.metrics import classification_report

print(classification_report(y_test_bow, prediction_float))

              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         2
         1.0       0.86      1.00      0.92        12

    accuracy                           0.86        14
   macro avg       0.43      0.50      0.46        14
weighted avg       0.73      0.86      0.79        14



C:\Users\hlinh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hlinh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\hlinh\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [165]:
X_train_tfidf, X_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(tfidf, y, random_state=42, test_size=0.2)
lreg_tfidf = LogisticRegression()
lreg_tfidf.fit(X_train_tfidf, y_train_tfidf) # training the model
prediction = lreg_tfidf.predict_proba(X_test_tfidf) # predicting on the validation set
prediction_float = prediction[:,1] >= 0.3 # if prediction >= 0.2 then 1 or else 0
prediction_float = prediction_float.astype('float')
f1_score(y_test_tfidf, prediction_float) # calculate f1 score

0.923076923076923